# TITAN-TEXT-EXPRESS-V1

In [18]:
import boto3
import json
brt = boto3.client(service_name='bedrock-runtime', region_name="us-east-1")

message = "hi, how are you?\n"

# system_message = '''You are hired as a data scientist on a new VALORANT esports team and 
#     have been tasked by the team’s general manager to support the scouting and recruitment 
#     process, please answer the above question.
#     '''
# message += system_message


# The request body for titan-text-express-v1
body = json.dumps({
    "inputText": message,
    "textGenerationConfig": {
        "maxTokenCount": 2048,
        "stopSequences": ["User:"],
        "temperature": 0,
        "topP": 0.9
    }
})
modelId = 'amazon.titan-text-express-v1'

accept = 'application/json'
contentType = 'application/json'

response = brt.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

response_body = json.loads(response.get('body').read())

output_text = response_body['results'][0]['outputText']

output_text

'Hello! How can I help you?'

# LLAMA3-8B-INSTRUCT

In [25]:
import json
import logging
import boto3


model_id = "meta.llama3-8b-instruct-v1:0"
prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for travel tips and recommendations<|eot_id|><|start_header_id|>user<|end_header_id|>

What can you help me with?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
max_gen_len = 512
temperature = 0.1
top_p = 0.9

# Create request body.
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": max_gen_len,
    "temperature": temperature,
    "top_p": top_p
})

bedrock = boto3.client(service_name='bedrock-runtime', region_name="us-east-1")
response = bedrock.invoke_model(
    body=body, modelId=model_id)

response_body = json.loads(response.get('body').read())
    
response_body

{'generation': "\n\nI'd be delighted to assist you with your travel plans! I can help you with a wide range of topics, including:\n\n1. Destination recommendations: I can suggest popular and off-the-beaten-path destinations based on your interests, budget, and preferences.\n2. Itinerary planning: I can help you plan your trip, including creating a daily schedule, recommending activities, and suggesting the best places to eat and stay.\n3. Travel tips and advice: I can provide you with insider tips and advice on how to navigate airports, public transportation, and foreign cultures.\n4. Packing lists: I can help you create a packing list tailored to your destination and activities.\n5. Language and cultural insights: I can provide you with basic phrases, cultural norms, and customs to help you navigate foreign languages and cultures.\n6. Budgeting and cost-saving tips: I can offer suggestions on how to save money on flights, accommodations, and activities.\n7. Travel documentation: I can

In [26]:
response_body['generation']

"\n\nI'd be delighted to assist you with your travel plans! I can help you with a wide range of topics, including:\n\n1. Destination recommendations: I can suggest popular and off-the-beaten-path destinations based on your interests, budget, and preferences.\n2. Itinerary planning: I can help you plan your trip, including creating a daily schedule, recommending activities, and suggesting the best places to eat and stay.\n3. Travel tips and advice: I can provide you with insider tips and advice on how to navigate airports, public transportation, and foreign cultures.\n4. Packing lists: I can help you create a packing list tailored to your destination and activities.\n5. Language and cultural insights: I can provide you with basic phrases, cultural norms, and customs to help you navigate foreign languages and cultures.\n6. Budgeting and cost-saving tips: I can offer suggestions on how to save money on flights, accommodations, and activities.\n7. Travel documentation: I can help you under

In [23]:
response_body = json.loads(response.get('body').read())
response_body

{'inputTextTokenCount': 7,
 'results': [{'tokenCount': 9,
   'outputText': 'Hello! How can I help you?',
   'completionReason': 'FINISH'}]}

In [26]:
# 从response_body中获取outputText
output_text = response_body['results'][0]['outputText']
output_text

'Hello! How can I help you?'

In [38]:
import boto3
import json
brt = boto3.client(service_name='bedrock-runtime')

message = "hi, how are you?\n"

system_message = '''You are hired as a data scientist on a new VALORANT esports team and 
    have been tasked by the team’s general manager to support the scouting and recruitment 
    process, please answer the above question.
    '''

message += system_message

body = json.dumps({
    "inputText": message,
    "textGenerationConfig": {
        "maxTokenCount": 2048,
        "stopSequences": ["User:"],
        "temperature": 0,
        "topP": 0.9
    }
})

modelId = 'amazon.titan-text-express-v1'
accept = 'application/json'
contentType = 'application/json'

response = brt.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)

stream = response.get('body')
if stream:
    for event in stream:
        chunk = event.get('chunk')
        print(chunk)
        if chunk:
            print(json.loads(chunk.get('bytes')).get('outputText'), end='')


The VALORANT esports team is looking to hire a data scientist to support the scouting and recruitment process. The general manager has tasked the data scientist with providing insights and analysis to help the team identify and recruit talented players. The data scientist will work closely with the team’s coaches and management to gather and analyze player data, including game statistics, performance metrics, and video footage. They will use advanced analytics techniques to identify patterns and trends in player performance and develop predictive models to help the team make informed decisions about player recruitment. The data scientist will also be responsible for developing and maintaining a database of player information and statistics, which will be used by the team’s coaching staff and management to track player progress and make adjustments to the team’s strategy. In addition to their technical skills, the data scientist will need to have strong communication and interpersonal 